In [1]:
import numpy as np
import pandas as pd
import logging as log
import plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
plotly.plotly.sign_in('spersad', 'oNkuP1yzbpN734Ag8M9P')
import plotly.graph_objs as go

log.getLogger().setLevel(log.INFO)

## Simulate DMS modification on a given molecule

TODO: Detail assumptions in simulating DMS modification

In [11]:
def simulate_modification(structure, sequence, DMS_prob, n_molecules):
    '''
    Simulate the DMS modification to a given RNA structure, producing the resulting DMS modified bases
    @param: structure- dot/bracket notation indication an open/closed base
    @param: sequence - nucleotide sequence corresponding to the given RNA structure
    @param: DMS_prob - probability of mutating an open (A/C) nucleotide
    @param: n_molecules - number of molecules to simulate
    
    @return: molecules - an array of DMS and RT'ed molecules, eg ref=ATCG, mol=ATGG
    @return: reads - an array of molecules where 0 if no mutation, 1 if mutated, eg 0010
    @return: basevectors - an array of molecules where 0 if no mutation, otherwise the inserted base is present, eg 00G0
    '''
    
    structure=list(structure)
    sequence=list(sequence)
    assert len(structure)==len(sequence), 'Sequence and structure must have the same length'
    
    molecules = []
    reads = []
    basevectors = []
    for mol_num in range(n_molecules):
        molecule = []
        read = []
        basevector = []
        for (i,pos) in enumerate(structure):
            base = sequence[i]
            if base in ['A','C'] and pos == '.': #pos=='.' indicates the base is open 
                # For a candidate base, it is DMS modified with a given probability
                if np.random.random()<DMS_prob:
                    mutation = mutate_base(base)
                    molecule.append(mutation)
                    if mutation == base:  
                        # If mutated base is the same as the original base, we cannot detect it.
                        read.append(0)
                        basevector.append('0')
                    else: 
                        read.append(1)
                        basevector.append(base)
                else:
                    molecule.append(base)
                    read.append(0)
                    basevector.append('0')
            else:
                molecule.append(base)
                read.append(0)
                basevector.append('0')
        molecules.append(molecule)
        reads.append(read)
        basevectors.append(''.join(basevector))
    
    molecules = np.array(molecules)
    reads = np.array(reads)
    
    return molecules, reads, np.array(basevectors)
                

def mutate_base(original):
    ''' 
    Mutate a DMS modified base according to RT enzyme
    @param: original - original base
    @return: choice - modified base
    '''
    
    # Naive mutation distribution, estimate better from data 
                # A      T     C    G
    mut_dist = [[0.25, 0.25, 0.25, 0.25], #A
                [0.25, 0.25, 0.25, 0.25]] #C
    
    bases = {'A':0, 'C':1}
    prob_dist = mut_dist[bases[original]]
    choice = np.random.choice(['A','T','C','G'], p=prob_dist)
    return choice

def illegal_reads(reads):
    '''
    Compute the proportion of reads which are illegal (have two mutations within distance 3 of each other)
    @param: reads - array of n bitvectors (1 is mut, 0 is WT)
    @return: illegal - array of length n, where illegal[i] is 1 if read i is illegal, otherwise 0
    '''
    def check_read(read):
        dist_3 = read+np.concatenate(([0],read[:-1]))+np.concatenate(([0,0],read[:-2]))
        if 2 in dist_3:
            return 1
        return 0
    illegal = np.apply_along_axis(check_read,1,reads)
    return illegal

## Create bitvector

Wrapper for simulating multiple structures and creating bitvector files for each.

In [58]:
def simulate_reads(sequence, structures, struc_proportions, DMS_prob, n_reads, fname='TestFile'):
    '''
    Simulates the DMS modification of structures and returns a bitvector and simulated molecules DataFrame. 
    
    In the case of producing paired end reads, generates one bitvector file and two fasta files (one for each direction).

    Fasta file format example:
    >SRR041655.1 HWI-EAS284_61BKE:6:1:2:1735/1
    NAAATCAGACAAATCTCCGTTATTGGTATATACTTTGGGAGTGTTATGGAATTGCACACCCATTTCGAACATGAAGCCAATTCGTTTCTTAGGAATCGCT.
    
    @param: sequence - nucleotide sequence corresponding to the given RNA structure
    @param: structures - list of strings where each string is a dot bracket representation of an RNA structure. All structures must match the length of the sequence
    @param: struc_proportions - the proportions of each structure in the final bitvector file
    @param: DMS_prob - probability of mutating an open (A/C) nucleotide
    @param: n_reads - number of reads to simulate
    @param: fname - name to save bitvector file as
    
    @output: reads - DataFrame containing 'Query_name','Bases_vector','Molecueles',N_occur','N_mutations','N_deletions','Coverage','Reference','Index'
    
    @return: None. 
    '''
    read_length = len(sequence)
    assert len(structures)==len(struc_proportions),'The number of structures and the number of mixing proportions is not equal'
    #TODO:
    assert True,'The lengths of all structures must match the length of the sequence specified.'
    struc_proportions = np.array(struc_proportions)
    if sum(struc_proportions)!=1:
        print('WARNING: The structure proportions do not sum to 1. Normalizing proportions.')
        struc_proportions /= sum(structure_proportions)
    
    for i,struc in enumerate(structures):
        print('Simulating structure {0} of {1}'.format(i+1, len(structures)))
        prop = struc_proportions[i]
        mols, reads, basevectors = simulate_modification(struc, sequence, DMS_prob, int(n_reads*prop))
        illegal = illegal_reads(reads)
        percent = 100*np.sum(illegal)/len(illegal)
        print('{0} illegal reads: {1}%'.format(np.sum(illegal),percent))
        good_reads = reads[illegal==0]
        good_bases = basevectors[illegal==0]
        good_molecules = mols[illegal==0]
        
        num_reads = len(good_bases)
        (good_bases, idx, counts) = np.unique(good_bases, return_index=True, return_counts=True)
        num_unique = len(good_bases)
        good_molecules = good_molecules[[idx]]
        good_molecules = [''.join(mol) for mol in good_molecules]
        
        df_dict = {'Query_name':list(range(num_unique)),'Bases_vector':good_bases,'Molecules':good_molecules,'N_occur':counts, 
                   'N_mutations':[0]*num_unique ,'N_deletions':[0]*num_unique ,'Coverage':[read_length]*num_unique ,
                   'Reference':['struc_{0}'.format(i+1)]*num_unique, 'Index':['']*num_unique}
        pd.DataFrame(df_dict).to_csv('{0}_{1}_reads.txt'.format(fname,i+1),sep='\t',index=False, columns=['Query_name','Bases_vector','Molecules','N_occur','N_mutations','N_deletions','Coverage','Reference','Index'])
        
    print('Reads generated.')
    return good_molecules, idx

In [59]:
struc = '((((.(((((((((((.(((((...((((......)))))))))))))))))))))))).((((.....(((((.(((..(((.(((..((((((....((....(((((((((((.(.......((((((...)))))).(((....(((.......))).....)))..................(((.....)))..................................(((.(((....((((.(.(((((.........)))))...).))))..))).))).(((((....))))).....(((..(((((....)))))..))).).)))))))))))...))...))))))....))).)))..))).........(((....)))((((.(((((((((....................................................................)))))))))................((((((...........((((............))))...............................................((((((((((((((.((((........)))).................(((((.....................................)))))...((.....)).((((((...((.........))..)))))).............................((((....((((...............................(((((((((.................(((.....))).........)))))))))...........((....)).)))))))).......................))))))).))))))).....)))))).................)))).....))))).....))))....((((.......)))).'
seq= 'GGGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTCAAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACTTGAAAGCGAAAGTAAAGCCAGAGGAGATCTCTCGACGCAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCGGTATTAAGCGGGGGAGAATTAGATAAATGGGAAAAAATTCGGTTAAGGCCAGGGGGAAAGAAACAATATAAACTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTTTTAGAGACATCAGAAGGCTGTAGACAAATACTGGGACAGCTACAACCATCCCTTCAGACAGGATCAGAAGAACTTAGATCATTATATAATACAATAGCAGTCCTCTATTGTGTGCATCAAAGGATAGATGTAAAAGACACCAAGGAAGCCTTAGATAAGATAGAGGAAGAGCAAAACAAAAGTAAGAAAAAGGCACAGCAAGCAGCAGCTGACACAGGAAACAACAGCCAGGTCAGCCAAAATTACCCTATAGTGCAGAACCTCCAGGGGCAAATGGTACATCAGGCCATATCACCTAGAACTTTAAATGCATGGGTAAAAGTAGTAGAAGAGAAGGCTTTCAGCCCAGAAGTAATACCCATGTTTTCAGCATTATCAGAAGGAGCCACCCCACAAGATTTAAATACCATGCTAAACACAGTGGGGGGACATCAAGCAGCCATGCAAATGTTAAAAGAGACCATCAATGAGGAAGCTGCAGAATGGGATAGATTGCATCCA'
m,i = simulate_reads(seq, [struc], [1], 0.1, 10000, fname='LongTestFile')